# In this file I attempt to build the basic framework for doing numerics on the Hyzenberg Spin Chain (without refrence to my professors work)
- Im doing feromagnet bc its cooler

# Imports

In [30]:
import numpy as np

# General variables

In [31]:
L = 120 # number of spins 
#(note: if you are going to be working with a spiral with periodic boundy condtions, make sure L is st the spiral is maintained through the Lth --> 0th spin)

J = 1 # energy factor

J_vec = J * np.array([np.random.choice([-1, 1]), np.random.choice([-1, 1]), 1])

# Defining the initial state

In [32]:
# Making a state wherein the initial spins are all in random directions
def make_random_state(n=L):
    state = [] # states are numpy arrays of 3D vecotors (numpy arrays)
    for i in range(n):
        spin_i = np.random.rand(3)
        state.append(spin_i / np.linalg.norm(spin_i))
    
    return np.array(state)

# Making a state wherin all the spins are along a certain z _dir
def make_uniform_state(n=L, z_dir=1):
    state = [] # states are numpy arrays of 3D vecotors (numpy arrays)
    for i in range(n):
        spin_i = [0, 0, z_dir]
        state.append(spin_i)
    
    return np.array(state)

# Make a state wherin the spins look like they are tumbling over each other 
# (most likley the pi/4 spiral will be most used becuase we know it to be an unstable stationary state)
def make_spiral_state(n=L, spiral_angle=np.pi/4, phi=0):
    state = [] # states are numpy arrays of 3D vecotors (numpy arrays)
    for i in range(n):
        spin_i = [0, np.cos(i * spiral_angle + phi), np.sin(i * spiral_angle + phi)]
        state.append(spin_i)
    
    return np.array(state)

# The equation of motion

In [33]:
def S_dot(state):
    dstate_dt = []
    for i in range(len(state)):
        prev_index = i-1
        next_index = (i+1) % len(state) # This is assuming we are considering everything to be periodic
        dstate_dt.append(np.cross(-(J * state[prev_index] + J * state[next_index]), state[i]))

    return dstate_dt

## Testing S_dot for pi/4 spin state

In [34]:
test_state = make_spiral_state(L)

print(S_dot(test_state))

[array([-5.77315973e-15,  0.00000000e+00,  0.00000000e+00]), array([1.11022302e-16, 0.00000000e+00, 0.00000000e+00]), array([-2.44266968e-17,  0.00000000e+00,  0.00000000e+00]), array([-1.11022302e-16,  0.00000000e+00,  0.00000000e+00]), array([6.21689088e-17, 0.00000000e+00, 0.00000000e+00]), array([0., 0., 0.]), array([3.77422119e-17, 0.00000000e+00, 0.00000000e+00]), array([0., 0., 0.]), array([1.33155151e-17, 0.00000000e+00, 0.00000000e+00]), array([1.11022302e-16, 0.00000000e+00, 0.00000000e+00]), array([-5.66222694e-16,  0.00000000e+00,  0.00000000e+00]), array([1.11022302e-15, 0.00000000e+00, 0.00000000e+00]), array([7.54844239e-17, 0.00000000e+00, 0.00000000e+00]), array([-1.22124533e-15,  0.00000000e+00,  0.00000000e+00]), array([-5.99645754e-17,  0.00000000e+00,  0.00000000e+00]), array([1.22124533e-15, 0.00000000e+00, 0.00000000e+00]), array([2.66310302e-17, 0.00000000e+00, 0.00000000e+00]), array([-1.22124533e-15,  0.00000000e+00,  0.00000000e+00]), array([2.20433337e-18, 0